In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Function calling in parallel

This notebook simplifies the below original notebook written by Kristopher Overholt
[Working with Parallel Function Calls and Multiple Function Responses in Gemini](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling.ipynb#scrollTo=VEqbX8OhE8y9)

* [Parallel function calling example](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling#parallel-samples)

## Overview

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

## Install Vertex AI SDK

In [1]:
%pip install --upgrade --quiet --user google-cloud-aiplatform \
                                      wikipedia

## Set configuration

### Authenticate to access to the GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab execution

### Define constants

In [17]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [18]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession,
    FunctionDeclaration
)

from vertexai.preview.generative_models import (
    grounding,
)
from vertexai.preview import (
    caching
)

#----

from typing import Any
import wikipedia


### Initalize Vertex AI

In [19]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Define helper function

In [20]:
# Helper function to extract one or more function calls from a Gemini Function Call response

def extract_function_calls(response: GenerationResponse) -> list[dict]:
    """
    Function to extract one or more function calls from a Gemini Function Call response

    Args:
        response: The Gemini Function Call response

    Returns:
        A list of function name

    """
    function_calls: list[dict] = []

    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            function_call_dict: dict[str, dict[str, Any]] = {function_call.name: {}}
            for key, value in function_call.args.items():
                function_call_dict[function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls



## Multiple function calls with a single functions

### Write function declarations and wrap them in a tool


In [27]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
    ],
)

### Initialize the Gemini model

In [28]:
model = GenerativeModel(
    MODEL_NAME,
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

In [29]:
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"
response = chat.send_message(prompt)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "solar panels"
            }
          }
        }
      }
    }
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "renewable energy"
            }
          }
        }
      }
    }
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "battery storage"
            }
          }
        }
      }
    }
  }
  avg_logprobs: -5.1288529195719297e-05
  finish_reason: STOP
}
model_version: "gemini-1.5-flash-002"
usage_metadata {
  prompt_token_count: 36
  candidates_token_count: 18
  total_token_count: 54
}

### Extract function names and parameters

In [30]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'solar panels'}},
 {'search_wikipedia': {'query': 'renewable energy'}},
 {'search_wikipedia': {'query': 'battery storage'}}]

### Define function to search

In [31]:
def search_wikipedia(function_calls):

    # Make external API call
    result = wikipedia.summary(function_call["search_wikipedia"]["query"])

    return result

### Call the function three times

In [32]:
api_response = []

# Loop over multiple function calls
for function_call in function_calls:

    result = search_wikipedia(function_calls)

    # Make external API call
    # result = wikipedia.summary(function_call["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

api_response

['A solar panel is a device that converts sunlight into electricity by using photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. These electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels are also known as solar cell panels, solar electric panels, or PV modules.\nSolar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms or rooftop solar panels which  supply the electricity grid\nSome advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages are that they depend o

### Get a natural language summary

In [33]:
# Return the API response to Gemini
response = chat.send_message(
    [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[0],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[1],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[2],
            },
        ),
    ],
)
display(Markdown(response.text))

Based on my research of Wikipedia articles, here's a summary of solar panels, renewable energy, and battery storage:

**Solar Panels:** Solar panels convert sunlight into electricity using photovoltaic (PV) cells.  They are often arranged in arrays and can be used for residential, commercial, and industrial purposes.  Advantages include using a clean energy source and reducing electricity bills; disadvantages include dependence on sunlight and high initial costs.  Solar panels are frequently used in conjunction with battery storage.

**Renewable Energy:** Renewable energy comes from natural resources that replenish quickly.  Common types include solar, wind, hydro, bioenergy, and geothermal.  It's a key strategy to combat climate change by reducing greenhouse gas emissions and air pollution.  While costs have decreased significantly, making it competitive with fossil fuels, challenges remain, such as land use and mineral extraction impacts.

**Battery Storage:** Battery energy storage systems (BESS) store electrical energy from various sources, including renewable energy like solar and wind.  They help stabilize power grids by providing quick responses to fluctuations in supply and demand, preventing outages.  While currently smaller in scale than other storage methods like pumped hydro, BESS is rapidly growing and becoming more cost-effective.  The technology is crucial for integrating intermittent renewable energy sources into the grid.


## Parallel function calls across multiple functions

In [34]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

suggest_wikipedia = FunctionDeclaration(
    name="suggest_wikipedia",
    description="Get suggested titles from Wikipedia for a given term",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for suggested titles on Wikipedia",
            },
        },
    },
)

summarize_wikipedia = FunctionDeclaration(
    name="summarize_wikipedia",
    description="Get article summaries from Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "Query to search for article summaries on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
        suggest_wikipedia,
        summarize_wikipedia,
    ],
)

### Initialize the Gemini model


In [36]:
model = GenerativeModel(
    MODEL_NAME,
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini


In [37]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

response = chat.send_message(prompt)

### Extract function names and parameters

In [39]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'solar system'}},
 {'suggest_wikipedia': {'query': 'solar system'}},
 {'summarize_wikipedia': {'topic': 'solar system'}}]

### Make external API calls

In [40]:
api_response: dict[str, Any] = {}  # type: ignore

# Loop over multiple function calls
for function_call in function_calls:
    print(function_call)
    for function_name, function_args in function_call.items():
        # Determine which external API call to make
        if function_name == "search_wikipedia":
            result = wikipedia.search(function_args["query"])
        if function_name == "suggest_wikipedia":
            result = wikipedia.suggest(function_args["query"])
        if function_name == "summarize_wikipedia":
            result = wikipedia.summary(function_args["topic"], auto_suggest=False)

        # Collect all API responses
        api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}}
{'suggest_wikipedia': {'query': 'solar system'}}
{'summarize_wikipedia': {'topic': 'solar system'}}


### Get a natural language summary

In [41]:
# Return the API response to Gemini
response = chat.send_message(
    [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response.get("search_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="suggest_wikipedia",
            response={
                "content": api_response.get("suggest_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="summarize_wikipedia",
            response={
                "content": api_response.get("summarize_wikipedia", ""),
            },
        ),
    ],
)

display(Markdown(response.text))

Based on the available tools, here's a summary of information about the solar system from Wikipedia:

**Search Results:** The search for "solar system" on Wikipedia yields results including articles on the Solar System itself, its formation, lists of objects within it, and related disambiguation pages.

**Variations:**  The only variation suggested by the API was "soler system," which is likely a misspelling.

**Summary:** The Wikipedia summary describes the solar system as a gravitationally bound system comprising the Sun and the objects orbiting it.  It details the formation of the solar system approximately 4.6 billion years ago, the Sun's characteristics as a G-type main-sequence star, and the classification of the eight planets (four terrestrial and four giant planets).  The summary also mentions dwarf planets, other small bodies (asteroids, comets, etc.), and the heliosphere, which is the boundary of the solar system's influence.  Finally, it notes the distance to the nearest star, Proxima Centauri.
